<a href="https://colab.research.google.com/github/pawanaditya85/FinChat/blob/main/FinChat_Finetuned.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
pip install rouge-score nltk

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=541e7028593c2e095400fc497acbe49fcf6173fa6054ad582ad1dc6e175d3067
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


# **Importing Modeling and Testing**

In [1]:
import openai
my_sk = "FN_SK"
openai.api_key = my_sk
import time

In [2]:
from openai import OpenAI
client = OpenAI(api_key=my_sk)

Libraries for data preparation

In [3]:
import csv
import json
import random

# **Prepaing Training Data**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
question_list= []
answer_list= []

with open('/content/drive/MyDrive/data/qa_dataset.csv', mode='r') as file:
  file = csv.reader(file)

  for line in file:
    if line[0]=='input':
      continue

    question_list.append(line[0])
    answer_list.append(line[1] + " -FinChat")

Conveting the CSV format data to JSONL format for model finetuning (which is a required format mentioned by OPEN AI)

In [15]:
example_list=[]

instructions_string ="""FinChat is a AI chat assistant primarly focusing on personal finance advices for users, addresses user queries on topics ranging from investment strategies, Budgeting, Saving, Retirement Planning, Insurance and Risk Management,Credit and Debt Management. \
It responces to questions fom user and concludes with its signrature ' -FinChat'. \
It uses data which was used for fine tuning the model to answer and question for better responce\

Here are examples of FinChat responding to user questions.

input: What's a good strategy for saving for retirement?
output: A good retirement saving strategy involves starting early, maximizing your contributions to retirement accounts like a 401(k) or IRA, and investing in a diversified portfolio. For example, if you start investing $200 a month from age 25 in a retirement account with an average annual return of 7%, you could accumulate over $500,000 by the time you retire at 65. -FinChat

input: How can I improve my credit score?
output: Improving your credit score can be achieved by consistently paying your bills on time, reducing your debt-to-credit ratio, and avoiding opening new credit lines unnecessarily. For instance, paying down a credit card balance that is close to the credit limit can have a noticeable impact on your credit score. -FinChat

input: Should I choose a fixed or variable rate mortgage?
output: Choosing between a fixed and variable rate mortgage depends on your financial stability and risk tolerance. A fixed-rate mortgage is safer if you prefer predictable payments, while a variable rate might initially be lower but can increase over time. For example, during periods of low interest rates, a fixed rate might help lock in savings long-term. -FinChat

input: What are some effective budgeting techniques?
output: Effective budgeting techniques include the 50/30/20 rule, where 50% of your income goes towards necessities, 30% towards wants, and 20% towards savings and debt repayment. Additionally, tracking your expenses using apps can help identify areas to cut back. Real-life example: Categorizing your expenses can reveal unnecessary subscriptions or frequent dine-outs that can be reduced. -FinChat

input: How should I manage risk when investing?
output: Managing risk when investing involves diversifying your portfolio across different asset classes and industries. For instance, mixing stocks, bonds, and real estate investments can protect against market volatility. Also, consider your investment horizon and risk tolerance to align your portfolio accordingly. -FinChat

input: What’s the best way to manage debt effectively?
output: Managing debt effectively can be approached by prioritizing high-interest debts first, a method known as the avalanche technique. Additionally, consolidating multiple debts into a single loan with a lower interest rate can reduce monthly payments and simplify finances. For example, consolidating credit card debt through a personal loan might reduce interest rates from around 20% to under 10%. -FinChat"""


for i in range(len(question_list)):
  system_dict = {"role": "system", "content": instructions_string}
  user_dict = {"role": "user", "content": question_list[i]}
  assistant_dict = {"role": "assistant", "content": answer_list[i]}

  messages_list = [system_dict, user_dict, assistant_dict]

  example_list.append({"messages": messages_list})

Creating Train/validation split

In [7]:
validation_index_list = random.sample(range(0, len(example_list)-1), 200)

validation_data_list = [example_list[index] for index in validation_index_list]

for example in validation_data_list:
  example_list.remove(example)

In [8]:
print(len(example_list))
print(len(validation_data_list))

1787
200


Writing Example to JSONL files

In [9]:
with open('/content/drive/MyDrive/data/training_data.jsonl', 'w') as training_file:
  for example in example_list:
    json.dump(example, training_file)
    training_file.write('\n')

with open('/content/drive/MyDrive/data/validation_data.jsonl', 'w') as validation_file:
  for example in validation_data_list:
    json.dump(example, validation_file)
    validation_file.write('\n')

uploading Training data to OpenAI API

In [10]:
training_file = client.files.create(
    file = open('/content/drive/MyDrive/data/training_data.jsonl', 'rb'),
    purpose = "fine-tune"
)

validation_file = client.files.create(
    file = open('/content/drive/MyDrive/data/validation_data.jsonl', 'rb'),
    purpose = "fine-tune"
)

# Create a Fine-tuned Model

In [ ]:
client.fine_tuning.jobs.create(
    training_file = training_file.id,
    validation_file = validation_file.id,
    suffix = "FinChat",
    model = "gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-86j55h50hIVp9HmJLqBifwpL', created_at=1732618188, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-nIsl9crHa1Sd0m8Zi5P4GYvc', result_files=[], seed=896319142, status='validating_files', trained_tokens=None, training_file='file-UeqVqV5dKUrRj5QwCz6Lg8', validation_file='file-7FPhXSUDCMzhZCXHBEZywn', estimated_finish=None, integrations=[], user_provided_suffix='FinChat')

# Testing the Fine Tuned Model

In [ ]:
test_question = "What are some effective budgeting techniques?"

In [ ]:
response = client.chat.completions.create(
    model="ft:gpt-3.5-turbo-0125:personal:finchat:AXoAnqkh",
    messages=[
    {"role": "system", "content": instructions_string},
    {"role": "user", "content": test_question}
    ]
)

In [ ]:
print(dict(response)['choices'][0].message.content)

Here's a variant suggestion assuming you're employed: Try to compute and maintain a comprehensive income statement that includes amortizations for bigger expenses. At the beginning of the month, when you compute your income, you should also add to it those amortizations to calculate an effective income, from which you'd calculate your spending limits. "Amortization" is a variation from what is known as "zero-based budgeting" where, in order to work out a budget, you sit down and try to write out a comprehensive list of things you want to (and have to) spend for, and you can only start the month after you've worked this out. The problem I see with that, is that if you need to spend $X/mo for as long as you can see into the future (e.g. you have to drive to work, etc.), then the derived $X/mo becomes the minimum you can ever manage with. The added complication is that your actual income tends to see some slight variation over the year, if bonuses come and go, you have vacation days, etc.

Evaluation Fine Tuned Model with

BLEU:

Measures the overlap of n-grams between the predicted and ground truth texts.
The sentence_bleu function computes BLEU for each prediction.

ROUGE:

ROUGE-1: Measures overlap of unigrams (single words).

ROUGE-2: Measures overlap of bigrams (two consecutive words).

ROUGE-L: Measures the longest common subsequence overlap.

In [18]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

# Example test dataset
test_dataset = [
    {"question": "How to save money on currency conversion", "expected_answer": "If you want to convert more than a few thousand dollars, one somewhat complex method is to have two investment accounts at a discount broker that operations both in Canada and the USA, then buy securities for USD on a US exchange, have your broker move them to the Canadian account, then sell them on a Canadian exchange for CAD.  This will, of course, incur trading fees, but they should be lower than most currency conversion fees if you convert more than a few thousand dollars, because trading fees typically have a very small percentage component. Using a currency ETF as the security to buy/sell can eliminate the market risk. In any case, it may take up to a week for the trades and transfer to settle."},
    {"question": "Do I need to file a tax return as a student?", "expected_answer": "In the U.S., Form 1040 is known as the tax return.  This is the form that is filed annually to calculate your tax due for the year, and you either claim a refund if you have overpaid your taxes or send in a payment if you have underpaid.  The form is generally due on April 15 each year, but this year the due date is April 18, 2016. When it comes to filing your taxes, there are two questions you need to ask yourself: Am I required to file? The 1040 instructions has a section called with several charts that determine if you are legally required to file.  It depends on your status and your gross income.  If you are single, under 65, and not a dependent on someone else's return, you are not required to file if your 2015 income was less than $10,300.  If you will be claimed as a dependent on someone else's return, however, you must file if your earned income (from work) was over $6300, or your unearned income (from investments) was over $1050, or your gross (total) income was more than the larger of either $1050 or your earned income + $350.  See the instructions for more details. Should I file? Even if you find that you are not required to file, it may be beneficial to you to file anyway.  There are two main reasons you might do this: If you have had income where tax has been taken out, you may have overpaid the tax.  Filing the tax return will allow you to get a refund of the amount that you overpaid. As a student, you may be eligible for student tax credits that can get you a refund even if you did not pay any tax during the year. How to file For low income tax payers, the IRS has a program called Free File that provides free filing software options."},
]

# Initialize lists to store results
predictions = []
ground_truths = []

# Generate predictions
for test_case in test_dataset:
    question = test_case["question"]
    expected_answer = test_case["expected_answer"]

    response = client.chat.completions.create(
        model="ft:gpt-3.5-turbo-0125:personal:finchat:AXoAnqkh",
        messages=[
            {"role": "system", "content": "Answer the question based on financial principles."},
            {"role": "user", "content": question}
        ]
    )

    predicted_answer = dict(response)['choices'][0].message.content
    predictions.append(predicted_answer)
    ground_truths.append(expected_answer)

# Compute BLEU and ROUGE Scores
bleu_scores = []
rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = []

for pred, gt in zip(predictions, ground_truths):
    # BLEU score
    bleu = sentence_bleu(
        [gt.split()],
        pred.split(),
        smoothing_function=SmoothingFunction().method1
    )
    bleu_scores.append(bleu)

    # ROUGE score
    rouge = rouge_scorer_instance.score(gt, pred)
    rouge_scores.append(rouge)

# Calculate average scores
average_bleu = sum(bleu_scores) / len(bleu_scores)
average_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
average_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
average_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

# Print metrics
print(f"Average BLEU Score: {average_bleu:.4f}")
print(f"Average ROUGE-1 Score: {average_rouge1:.4f}")
print(f"Average ROUGE-2 Score: {average_rouge2:.4f}")
print(f"Average ROUGE-L Score: {average_rougeL:.4f}")


Average BLEU Score: 0.0184
Average ROUGE-1 Score: 0.3239
Average ROUGE-2 Score: 0.0519
Average ROUGE-L Score: 0.1270
